In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from sklearn.linear_model import LinearRegression

In [2]:
def name_changer(st) -> str:
    st = st.split('\\')  # C:\Users\user\!BMM\Digital_Breakthrough\NPO_dataset
    st = '/'.join(st)
    return st

def file_scanner(path:str) -> list:
    path = name_changer(path)
    names = os.listdir(path)
    names = names[:]
    def file_loader():
        data = []
        for name in names:
            im_frame = pd.read_csv(path  + "/" + name)
            typ_data = im_frame
            typ_data = typ_data.interpolate()   ##Дозаполнение таблицы и удаление строчек, где дозаполнение невозможно
            typ_data = typ_data.dropna()
            typ_data = typ_data.reset_index(drop=True)
            data.append(typ_data)    
        return data
    data = file_loader()
    return data
def to_ticks(time_raw : str):
    time_raw = time_raw.split()[0]
    t0 = datetime.fromisocalendar(1993, 1, 7)
    time = datetime.strptime(time_raw, '%Y-%m-%d')
    ticks = (time - t0).total_seconds()
    return ticks

In [13]:
Path = r"C:\Users\user\!BMM\Digital_Breakthrough\NPO_dataset"
print(os.listdir(Path))
data_files = file_scanner(Path)
data_files[0]

['npo_clnts.csv', 'npo_cntrbtrs.csv', 'out_time.csv']


,clnt_id,gndr,slctn_nmbr,age,brth_yr,pstl_code,city
0,0xD1930AC934CD0D4AB6141DF45637EFE4,1.0,2,74.0,1949.0,188544,0
1,0x25DCE99C94913C42A49F739DDA3AE81A,0.0,2,62.0,1961.0,398046,0
2,0xCF29021EFE24454693866565B7CAB0D8,1.0,0,69.0,1954.0,162609.0,0
3,0xA6D80BE13B6FEB4591C5E93F715C0B98,0.0,0,38.0,1984.0,162623,0
4,0x88C8CB57D2D6B14393894C0CBB8A9A4A,0.0,2,38.0,1984.0,173025,0
...,...,...,...,...,...,...,...
220180,0x001696C75F4BE1439203E5433FB5AC97,0.0,1,43.0,1980.0,192289.0,1
220181,0x1DB0353AE19D4F47B87C72BAFEED5671,1.0,1,52.0,1970.0,664047.0,0
220182,0xDB06023E99D7A44693A0669AB7FAB8E7,0.0,2,72.0,1951.0,356884,0
220183,0xE07E152B0243284A92FE34A0EC42AB5F,1.0,0,64.0,1958.0,302038,0


In [123]:
cl_info = data_files[0] #clients
acc = data_files[1] #bank accounts
trn = data_files[2] #transactions 
trn = trn.rename(columns={"628548768000000000": "time", " 20.32": "money", " 0":'type'}) #time 86400 * 10**7  ##kvartal = 91day
trn['time'] = trn['time'].map(lambda x: (x //(86400 * 10**7)) - 727487) 

In [203]:
trn['money'][2]

20.32

In [172]:
# min(trn['time'])
# trn.sort_values('time')
data_set = []
now_time = 400#trn['time'][-2:-1].tolist()[0]
for time in range(0, now_time, 15):  
    sv_idx = time               #ВОзвращение счётчика начала поиска в момент вермени
    kvartal = time + 91         #граница хода цикла
    income, outcome = 0,0       #обнуление прибыли и издержек
    for idx ,row in trn[sv_idx:].iterrows():
        if row[0] > kvartal:        #проверка периода времени, если выходит за рамки квартала - выходим из цикла
            break            
        sv_idx = idx
        money, opt = row[1], row[2]
        if money > 0 and opt == 0: income += money
        if money < 0 and opt == 0: outcome += money
        elif money > 0 and opt == 1: income += money
    cashflow = income - outcome
    print(time/now_time, time, now_time)
    data_set.append((income, outcome, cashflow))

0.0 0 400
0.0375 15 400
0.075 30 400
0.1125 45 400
0.15 60 400
0.1875 75 400
0.225 90 400
0.2625 105 400
0.3 120 400
0.3375 135 400
0.375 150 400
0.4125 165 400
0.45 180 400
0.4875 195 400
0.525 210 400
0.5625 225 400
0.6 240 400
0.6375 255 400
0.675 270 400
0.7125 285 400
0.75 300 400
0.7875 315 400
0.825 330 400
0.8625 345 400
0.9 360 400
0.9375 375 400
0.975 390 400


gndr = data[0]['gndr'].to_frame(name='gender')   #Преобразование таблиц
age = data[0]['age'].to_frame(name='age')                   #Получаем нужные данные из каждой таблицы, а затем собираем их в 2 удобные csv таблицы
brth_yr = data[0]['brth_yr'].to_frame(name='date')
pstl_code = data[0]['pstl_code'].to_frame(name='post_code')
first_out = pd.concat([gndr, age, brth_yr, pstl_code], axis=1)
first_out

npo_blnc = data[1]['npo_blnc'].to_frame(name='balance')
npo_pmnts_sum = data[1]['npo_pmnts_sum'].to_frame(name='sum')
npo_pmnts_nmbr = data[1]['npo_pmnts_nmbr'].to_frame(name='number')
npo_frst_pmnt_date = data[1]['npo_frst_pmnt_date'].to_frame(name='first_date')
npo_lst_pmnt_date = data[1]['npo_lst_pmnt_date'].to_frame(name='last_date')
npo_ttl_incm = data[1]['npo_ttl_incm'].to_frame(name='income')
second_out = pd.concat([npo_blnc, npo_pmnts_sum, npo_pmnts_nmbr, npo_frst_pmnt_date, npo_lst_pmnt_date, npo_ttl_incm], axis=1)
second_out




In [188]:
inc, out, cash = [], [],[]
for x,y,z in data_set:
    inc.append(x)
    out.append(y)
    cash.append(z)
inc = np.array(inc)
out = np.array(out)
cash = np.array(cash)

In [199]:
inc[1:]
out[1:]
cash[:-1]
# out

[232223.16000000003,
 315203.23000000266,
 402369.8800000052,
 467557.94000000815,
 616406.8300000129,
 677565.3100000124,
 736648.0100000149,
 798865.0500000161,
 849717.1400000189,
 903975.7200000221,
 1144766.0700000238,
 1201130.1000000234,
 1251646.9200000183,
 1305425.290000012,
 1376202.1900000079,
 1424518.4400000055,
 1782607.1500000132,
 1819756.1300000087,
 1841394.8000000087,
 1887499.9000000046,
 1933978.420000002,
 1967757.6599999983,
 2728902.949999988,
 2772452.429999971,
 2800584.469999956,
 2836508.2699999423]

In [ ]:
metrics.classification_report(y_test, test_pred)

In [230]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
cashflow = cash
# преобразуем данные в нужный формат
X = np.vstack((inc, out)).T
y = cashflow
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5, random_state=2)



In [234]:
from sklearn.ensemble import HistGradientBoostingClassifier
X_train, X_test = inc[:2000], inc[2000:]
y_train, y_test = cash[:2000], cash[2000:]
clf = HistGradientBoostingClassifier(max_iter=100).fit(X_train, y_train)
clf.score(X_test, y_test)

ValueError: Expected 2D array, got 1D array instead:
array=[ 232223.16        315203.23        402369.88000001  467557.94000001
  616406.83000001  677565.31000001  736648.01000001  798865.05000002
  849717.14000002  903975.72000002 1144766.07000002 1201130.10000002
 1251646.92000002 1305425.29000001 1376202.19000001 1424518.44000001
 1782607.15000001 1819756.13000001 1841394.80000001 1887499.9
 1933978.42       1967757.66       2728902.94999999 2772452.42999997
 2800584.46999996 2836508.26999994 2851734.89999993].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
from sklearn.linear_model import LogisticRegression 
# создаем и обучаем модель
model = LogisticRegression()
model.fit(X_train, y_train)

# теперь вы можете использовать модель для предсказания cashflow
# например, для income=[1] и outcome=[1]
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)
len(train_pred) == len(y_train)
print(metrics.classification_report(y_train, train_pred, digits=3))


In [101]:
for i, row in acc[:10].iterrows():
    print(i, row[4:], sep="\n\n")

0

npo_accnt_status                  1
npo_accnt_status_date    2001-05-14
npo_blnc                   10158.96
npo_pmnts_sum               2276.42
npo_pmnts_nmbr                  1.0
npo_frst_pmnt_date       2005-08-31
npo_lst_pmnt_date        2005-08-31
npo_ttl_incm                5638.83
Name: 0, dtype: object
1

npo_accnt_status                  0
npo_accnt_status_date    2018-10-30
npo_blnc                  27836.626
npo_pmnts_sum              230084.4
npo_pmnts_nmbr                 55.0
npo_frst_pmnt_date       2013-03-07
npo_lst_pmnt_date        2017-09-10
npo_ttl_incm                39875.3
Name: 1, dtype: object
2

npo_accnt_status                  0
npo_accnt_status_date    2014-01-20
npo_blnc                  45514.292
npo_pmnts_sum               7921.95
npo_pmnts_nmbr                  8.0
npo_frst_pmnt_date       2012-11-26
npo_lst_pmnt_date        2013-06-30
npo_ttl_incm                  207.5
Name: 2, dtype: object
3

npo_accnt_status                  0
npo_accnt_status_da

In [ ]:
client_id = data_files[1]['clnt_id'] == '0xC2B51FD4FE57F7479210FD7258DF5B0B'
a = data_files[1][client_id]
print(a)
for client_id in data_files[0]['clnt_id']:
    # client_data = data_files[1][client_id]
    a = data_files[1][data_files[1]['clnt_id'] == client_id]